In [3]:
from fractions import Fraction 
import matplotlib.pyplot as plt

def add_min_negative(A):  # Избавляемся от отрцательных элементов матрицы, изменится только значение игры.
    m = abs(min(0, min(map(min, A))))
    if m != 0:
        for i in range(len(A)):
            for j in range(len(A[i])):
                A[i][j] += m
        return A, m
    return A, 0 

def table_forming(A):
    m, n = A.shape  # размерность матрицы
    table = np.full((m + 1, n + m + 2), Fraction(0)) #создаём дополнительные строки и столбцы для симплекс-таблицы

    for i in range(m):
        for j in range(n):
            table[i][j] = A[i][j]  # копируем матрицу в таблицу
            
    for i in range(m):
        table[i][i + n] = 1 #значения добавочных переменных
        table[i][-2] = 1  #столбец значений 
        table[i][-1] = i + n + 1  #базис

    for i in range(n):
        table[-1][i] = -1  #в самом начале функция Z равна -1 на основных переменных

    return table


def step(table):
    m, n = table.shape 

    r, c, x = find_rcx(table) 
    for i in range(n-1):
        if x == 0:
            table[r][i] = 0
        else:
            table[r][i] = Fraction(table[r][i], x)
            
    for j in range(m):
        if j == r:
            continue
        mul = table[j][c]
        for i in range(n - 1):
            table[j][i] -= table[r][i] * mul  

    table[r][n - 1] = c + 1 
    return table



def find_rcx(table):  # Ищем строку и столбец
    m, n = table.shape  
    z_max = abs(min(table[-1])) 
    
    c = 0
    for i in range(n):
        if abs(table[-1][i]) == z_max:
            if table[-1][i] < 0:
                c = i
                
    found = False
    for j in range(m - 1):  
        if table[j][c] * table[j][-2] > 0: 
            
            new_val = Fraction(table[j][-2], table[j][c])
            
            if not found:
                found = True
                mtab = new_val
                r = j
            else:
                if new_val < mtab:
                    mtab = new_val
                    r = j

    return r, c, table[r][c]



def optimum(table):
    m, n = table.shape 
    second = np.full((n - m - 1), Fraction(0, 1)) 
    first = np.full((m - 1), Fraction(0, 1))
    
   
    for i in range(m - 1):  
        if table[i][-1] <= n - m - 1:
            second[table[i][-1] - 1] = table[i][-2]

    for i in range(m - 1):  
        first[i] = table[-1][i + n - m - 1]

    return second, first

def simplex(A):
    A, delta = add_min_negative(A)
    
    table = table_forming(A)
    while (table[-1] < 0).any(): 
        table = step(table)

    second, first = optimum(table)
    
    if sum(second) == 0:
        mul = 0
    else:
        mul = Fraction(1, sum(second))
    
    first *= mul
    second *= mul

    return second, first, mul - delta
    
def nash_equilibrium(A):                   
    p = [0] * len(A)
    q = [0] * len(A[0])
    pure = False

    for i in range(len(A)): # ищем седловую точку как нижнее и верхнее значение игры
        game_val = min(A[i])
        index_map = filter(lambda x: A[i][x] == game_val, range(len(A[i])))
        
        
        for j in list(index_map):
            for i in range(len(A)):
                if A[i][j] > game_val:
                    break
            else:
                p[i] = 1
                q[j] = 1
                pure = True
                return p, q, game_val, pure
                
    if not pure:
        p, q, game_val = simplex(A) #если седловой точки нет, ищем решение в смешанных стратегиях
        return p, q, game_val, pure

def spectr(p, name = ''):
    y = p
    x = np.arange(1, np.size(p) + 1)

    fig, ax = plt.subplots()
    
    ax.set_title("Спектры: " + name)
    ax.set_ylabel('Вероятности')
    ax.set_xlabel('Номер стратегии')

    ax.bar(x, y, width = 0.015, color = 'red' )
    ax.plot(x, y, marker='o', linestyle='', color = 'black')
    
def print_result(second, first, p, pure):

    print("Цена игры ", p)

    if not pure:
        print('Оптимальная смешанная стратегия 1 игрока | p | |', end="")
    else:
        print("                 Седловая точка \nПервый игрок  ", end="")

    for i in range(len(first)):
        print(str(first[i]).center(7), end='|')

    print()

    if not pure:
        print('\nОптимальная смешанная стратегия 2 игрока | q | |', end="")
    else:
        print("Второй игрок  ", end="")

    for i in range(len(second)):
        print(str(second[i]).center(7), end='|')
    
    spectr(first, name = "первый игрок")
    spectr(second, name = "второй игрок")


In [4]:
from fractions import Fraction
import numpy as np
import pytest


@pytest.mark.parametrize("matrix,p_ans,q_ans,price_ans,flag_ans", [
    (
        np.array([
            [4, 0, 6, 2, 2, 1],
            [3, 8, 4, 10, 4, 4],
            [1, 2, 6, 5, 0, 0],
            [6, 6, 4, 4, 10, 3],
            [10, 4, 6, 4, 0, 9],
            [10, 7, 0, 7, 9, 8]
        ]),
        np.array([Fraction(0, 1), Fraction(4, 31), Fraction(3, 31),
                  Fraction(27, 62), Fraction(21, 62), Fraction(0, 1)]),
        np.array([Fraction(0, 1), Fraction(0, 1), Fraction(257, 372),
                  Fraction(9, 62), Fraction(55, 372), Fraction(1, 62)]),
        Fraction(151, 31),
        False
    ),
    (
        np.array([
            [1, 0, 2, 0, 2],
            [2, 2, 2, 3, 0],
            [1, 5, 0, 0, 0],
            [0, 0, 1, 0, 3],
            [2, 1, 1, 3, 1]
        ]),
        np.array([Fraction(2, 7), Fraction(1, 7), Fraction(1, 7),
                  Fraction(1, 7), Fraction(2, 7)]),
        np.array([Fraction(3, 14), Fraction(3, 14), Fraction(9, 56),
                  Fraction(1, 28), Fraction(3, 8)]),
        Fraction(9, 7),
        False
    ),(
        np.array([
            [1,2,1,2],
            [2,1,2,4],
            [3,3,2,2],
            [4,1,3,3],
        ]),
        np.array([Fraction(0, 1), Fraction(0, 1), Fraction(2, 3), Fraction(1, 3)]),
        np.array([Fraction(0, 1), Fraction(1, 3), Fraction(1, 3), Fraction(1, 3)]),
        Fraction(7, 3),
        False
    ),
    (
        np.array([
            [1,2,1,2],
            [2,1,2,1],
            [3,2,3,2],
            [4,3,4,3],
        ]),
        np.array([Fraction(0, 1), Fraction(1, 1), Fraction(0, 1), Fraction(0, 1)]),
        np.array([Fraction(0, 1), Fraction(0, 1), Fraction(0, 1), Fraction(1, 1)]),
        Fraction(3, 1),
        True
    ),
    (
        np.array([
            [2,5,3],
            [3,1,7],
            [8,0,2],
        ]),
        np.array([Fraction(11, 17), Fraction(5, 34), Fraction(7, 34)]),
        np.array([Fraction(6, 17), Fraction(25, 68), Fraction(19, 68)]),
        Fraction(115, 34),
        False
    ),
    (
        np.array([
            [2,1,2],
            [1,2,1],
            [3,3,3],
        ]),
        np.array([Fraction(1, 1), Fraction(0, 1), Fraction(0, 1)]),
        np.array([Fraction(0, 1), Fraction(0, 1), Fraction(1, 1)]),
        Fraction(3, 1),
        True
    ),
    (
        np.array([
            [2,1,2],
            [1,2,1],
            [1,1,2],
        ]),
        np.array([Fraction(1, 2), Fraction(1, 2), Fraction(0, 1)]),
        np.array([Fraction(0, 1), Fraction(1, 2), Fraction(1, 2)]),
        Fraction(3, 2),
        False
    ),
    (
        np.array([
            [4,5,9,3],
            [8,4,3,7],
            [7,6,8,9],
        ]),
        np.array([Fraction(0, 1), Fraction(1, 1), Fraction(0, 1), Fraction(0, 1)]),
        np.array([Fraction(0, 1), Fraction(0, 1), Fraction(1, 1)]),
        Fraction(6, 1),
        True
    ),
    (
        np.array([
            [0,2,7],
            [12,11,1],
        ]),
        np.array([Fraction(11, 18), Fraction(7, 18)]),
        np.array([Fraction(1, 3), Fraction(0, 1), Fraction(2, 3)]),
        Fraction(14, 3),
        False
    ),
    (
        np.array([
            [6,5,7],
            [10,4,7],
            [13,10,4],
            [7,11,5],
        ]),
        np.array([Fraction(24, 35), Fraction(2, 35), Fraction(0, 1), Fraction(9, 35)]),
        np.array([Fraction(2, 35), Fraction(8, 35), Fraction(5, 7)]),
        Fraction(227, 35),
        False
    ),
])
def test_nash_test_equilibrium(matrix, p_ans, q_ans, price_ans, flag_ans):
    p_res, q_res, price_res, flag_res = nash_equilibrium(matrix)
    assert (p_res == q_ans).all()
    assert (q_res == p_ans).all()
    assert price_res == price_ans
    assert flag_res == flag_ans